In [4]:
! pip install transformers datasets

  Using cached datasets-3.5.1-py3-none-any.whl.metadata (19 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
Using cached datasets-3.5.1-py3-none-any.whl (491 kB)
Using cached dill-0.3.8-py3-none-any.whl (116 kB)
Using cached multiprocess-0.70.16-py311-none-any.whl (143 kB)
Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (194 kB)


### Import Libraries

In [1]:
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from transformers import Trainer
from transformers import TrainingArguments
from datasets import load_dataset
import torch

### Load the IMDb Dataset

In [2]:
dataset=load_dataset('imdb')
##it has train,text,unsupervised split

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

### Load Pre-trained BERT Tokenizer

In [3]:
tokenizer=BertTokenizer.from_pretrained("bert-base-uncased")

#load tokenize lowercase english,it will trun text into token IDs for BERT

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

### Tokenize the Dataset

In [4]:
## convert text into padded and truncated token IDs(token IDs >> subword into number)
# truncated >> cut the length means maxlength
# padding >> If a sentence is too short, we pad it with 0s so all inputs have the same length.
def tokenizer_function(examples):
    return tokenizer(examples["text"],padding="max_length",truncation=True)

#map(..batched=True) applies it to all examples efficiently
tokenized_dataset=dataset.map(tokenizer_function,batched=True)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

### Load the Pre-Trained BERT for Classification

In [5]:
#Binary classification (num_labels=2)
model=BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Prepare the Dataset for Training

In [6]:
# Remove the RAW Text ,convert to pytorch fromat for training
tokenized_datasets=tokenized_dataset.remove_columns(["text"])
tokenized_datasets.set_format("torch")

### Define Training Arguments

In [7]:
training_args = TrainingArguments(
  # Where to save model chechpoints
    output_dir="./results",
  # evaluate after each epoch
    evaluation_strategy="epoch",
  #mini-batch size for training
    per_device_train_batch_size=8,
  # how many time togo through the training data
    per_device_eval_batch_size=8,
    num_train_epochs=3,
  # regularization to avoid overfiting

    weight_decay=0.01,
  # save only last 2 checkpoints to save space
    logging_steps=10,
    save_total_limit=2,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Set Up the Trainer

In [8]:
trainer = Trainer(
    model=model,# our BERT model
    # args >> training settings.
    args=training_args,
    # Datasets for training and validation
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    # so that trainer can log inputs properly
    tokenizer=tokenizer,
)

<ipython-input-8-27f67eb6756e>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Fine-Tuning the model

In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

###Evaluate Accuracy after Training

In [ ]:
metrics=trainer.evaluate()
print(metrics)